## Using cuSpatial to Correlate Taxi Data after a Format Change
In 2017, the NYC Taxi data switched from giving their pickup and drop off locations in `lat/lon` to one of 262 `LocationID`s.  While these `LocationID`s made it easier to determine some regions and borough information that was lacking in the previous datasets, it made it difficult to compare datasets before and after this transition.  

By using cuSpatial `Points in Polygon` (PIP), we can now correlate the pick up and drop of latitudes and longitudes in a reasonable timeframe.  In this notebook, we will show you how to do so.  Currently, PIP only works on 32 polygons at once, so we will show how to process this larger 263 polygon shapefile with minimal memory impact.  Estimates are that it will take approximately 3-4x longer than if we didn't have that limitation, so something to look forward to!

You may need a 16GB card or larger.

## Imports

In [1]:
import cuspatial
import geopandas as gpd
import cudf
from numba import cuda
import numpy as np

## Download Data
We're going to download each January dataset months from 2015-2018. as well as the boundary.  The following cell, when uncommented will download just over 5GB of data.  If left uncommented, it will be just under 3GB.

In [2]:
!if [ ! -f "zones.zip" ]; then curl https://s3.amazonaws.com/nyc-tlc/misc/taxi_zones.zip -o zones.zip; else echo "zones.zip found"; fi
!if [ !  -f "taxi2015.csv" ]; then curl https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2015-01.csv -o taxi2015.csv; else echo "taxi2015.csv found"; fi
#!if [ !  -f "taxi2016.csv" ]; then curl https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2016-01.csv -o taxi2016.csv; else echo "taxi2016.csv found"; fi   
#!if [ !  -f "taxi2017.csv" ]; then curl https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2017-01.csv -o taxi2017.csv; else echo "taxi2017.csv found"; fi
!if [ !  -f "taxi2018.csv" ]; then curl https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2018-01.csv -o taxi2018.csv; else echo "taxi2018.csv found"; fi 
!if [ ! -f "taxi_zone_lookup.csv" ]; then curl https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv -o taxi_zone_lookup.csv; else echo "taxi_zone_lookup.csv found"; fi

zones.zip found
taxi2015.csv found
taxi2016.csv found
taxi2017.csv found
taxi2018.csv found
taxi_zone_lookup.csv found


## Read in the data
let's read in the spatial shape file and the taxi data.  While we won't use the look up data, we have it here for your reference

In [3]:
tzones = gpd.read_file('https://data.cityofnewyork.us/api/geospatial/d3c5-ddgc?method=export&format=GeoJSON')
tzones.to_file('cu_taxi_zones.shp')
taxi_zones = cuspatial.read_polygon_shapefile('cu_taxi_zones.shp')

In [4]:
taxi2015 = cudf.read_csv("taxi2015.csv")
taxi2018 = cudf.read_csv("taxi2018.csv")
taxi_zone_lookup = cudf.read_csv("taxi_zone_lookup.csv")

In [22]:
taxi2015.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,...,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,PULocationID,DOLocationID
0,2,2015-01-15 19:05:39,2015-01-15 19:23:42,1,1.59,-73.993896,40.750111,1,N,-73.974785,...,1,12.0,1.0,0.5,3.25,0.0,0.3,17.05,185,185
1,1,2015-01-10 20:33:38,2015-01-10 20:53:28,1,3.30,-74.001648,40.724243,1,N,-73.994415,...,1,14.5,0.5,0.5,2.00,0.0,0.3,17.80,212,212
2,1,2015-01-10 20:33:38,2015-01-10 20:43:41,1,1.80,-73.963341,40.802788,1,N,-73.951820,...,2,9.5,0.5,0.5,0.00,0.0,0.3,10.80,166,166
3,1,2015-01-10 20:33:39,2015-01-10 20:35:31,1,0.50,-74.009087,40.713818,1,N,-74.004326,...,2,3.5,0.5,0.5,0.00,0.0,0.3,4.80,230,230
4,1,2015-01-10 20:33:39,2015-01-10 20:52:58,1,3.00,-73.971176,40.762428,1,N,-74.004181,...,2,15.0,0.5,0.5,0.00,0.0,0.3,16.30,162,162


In [23]:
taxi2018.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,2018-01-01 00:21:05,2018-01-01 00:24:23,1,0.5,1,N,41,24,2,4.5,0.5,0.5,0.00,0.0,0.3,5.80
1,1,2018-01-01 00:44:55,2018-01-01 01:03:05,1,2.7,1,N,239,140,2,14.0,0.5,0.5,0.00,0.0,0.3,15.30
2,1,2018-01-01 00:08:26,2018-01-01 00:14:21,2,0.8,1,N,262,141,1,6.0,0.5,0.5,1.00,0.0,0.3,8.30
3,1,2018-01-01 00:20:22,2018-01-01 00:52:51,1,10.2,1,N,140,257,2,33.5,0.5,0.5,0.00,0.0,0.3,34.80
4,1,2018-01-01 00:09:18,2018-01-01 00:27:06,2,2.5,1,N,246,239,1,12.5,0.5,0.5,2.75,0.0,0.3,16.55


In [6]:
print(taxi_zones)

(0        1
1       34
2       35
3       36
4       37
      ... 
258    349
259    350
260    351
261    352
262    354
Name: f_pos, Length: 263, dtype: int32, 0        232
1       1113
2       1121
3       1137
4       1143
       ...  
349    97738
350    98010
351    98080
352    98162
353    98192
Name: r_pos, Length: 354, dtype: int32,                x          y
0     -74.184453  40.694996
1     -74.185632  40.691648
2     -74.185911  40.691450
3     -74.186370  40.691189
4     -74.187234  40.690595
...          ...        ...
98187 -73.937646  40.781001
98188 -73.937799  40.781031
98189 -73.937794  40.780781
98190 -73.937901  40.780750
98191 -73.938046  40.780830

[98192 rows x 2 columns])


## Correllating with cuSpatial Points In Polygon
Looking at the taxi zones and the taxi2015 data, you can see that
- 12.7 million pickup locations
- 12.7 million dropoff locations
- 263 LocationID features
- 354 LocationID rings
- 98,192 LocationID coordinates

Now that we've collected the set of pickup locations and dropoff locations, we can use `cuSpatial.point_in_polygon_bitmap` to quickly determine which pickups and dropoffs occur in each borough. That is, 353 LocationID rings composed of a total of 263 LocationID features.

To do this in a memory efficient way, instead of creating two massive 12.7 million x 263 arrays, we're going to use the 32 polygon limit to our advantage and map the resulting true values in the array a new `PULocationID` and `DOLocationID`, matching the 2018 schema.  Two things to note:
1. we had to go in a reversed order for this to work.  
1. locations outside of the `LocationID` areas are `264` and `265`.  We'll be using 264 to indicate our out-of-bounds zones as no guidance was given on how to decide between the two.

In [7]:
%%time
taxi2015['PULocationID'] = 264
taxi2015['DOLocationID'] = 264
for i in reversed(range(0,len(taxi_zones[0]), 32)):
    print(i, i+32)
    pickups = cuspatial.point_in_polygon_bitmap(taxi2015['pickup_longitude'] , taxi2015['pickup_latitude'], taxi_zones[0][i:i+32], taxi_zones[1], taxi_zones[2]['x'], taxi_zones[2]['y'])
    dropoffs = cuspatial.point_in_polygon_bitmap(taxi2015['dropoff_longitude'] , taxi2015['dropoff_latitude'], taxi_zones[0][i:i+32], taxi_zones[1], taxi_zones[2]['x'], taxi_zones[2]['y'])
    for j in range(i, i+32):
        try:
            taxi2015['PULocationID'][pickups[j]] = j
        except:
            pass
        try:
            taxi2015['DOLocationID'][pickups[j]] = j
        except:
            pass

256 288
224 256
192 224
160 192
128 160
96 128
64 96
32 64
0 32
CPU times: user 3min 27s, sys: 842 ms, total: 3min 28s
Wall time: 3min 28s


In [8]:
del pickups
del dropoffs

In [9]:
taxi2015['PULocationID'].value_counts()

237    467173
160    446675
236    441389
73     437823
229    429313
        ...  
109         3
26          2
45          1
104         1
203         1
Name: PULocationID, Length: 252, dtype: int32

## Verify
Let's test our outputs with a few `LocationID` values.

In [34]:
taxi2015.query('DOLocationID == 262')

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,...,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,PULocationID,DOLocationID,RatecodeID
18,2,2015-01-15 19:05:41,2015-01-15 19:20:36,1,2.37,-73.945541,40.779423,1,N,-73.980850,...,11.5,1.0,0.5,0.0,0.0,0.3,13.3,262,262,1
43,2,2015-01-25 00:13:06,2015-01-25 00:24:51,1,3.37,-73.945511,40.773724,1,N,-73.987434,...,12.5,0.5,0.5,0.0,0.0,0.3,13.8,262,262,1
122,1,2015-01-15 10:26:14,2015-01-15 10:42:16,1,1.90,-73.949486,40.772072,1,N,-73.970726,...,12.5,0.0,0.5,0.0,0.0,0.3,13.3,262,262,1
126,1,2015-01-15 10:26:15,2015-01-15 10:34:40,1,1.30,-73.947639,40.775143,1,N,-73.960693,...,7.5,0.0,0.5,1.0,0.0,0.3,9.3,262,262,1
138,2,2015-01-07 14:58:08,2015-01-07 15:06:36,1,0.98,-73.948692,40.777782,1,N,-73.955284,...,7.0,0.0,0.5,1.4,0.0,0.3,9.2,262,262,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12748825,1,2015-01-10 18:16:49,2015-01-10 18:26:59,4,0.90,-73.946709,40.772472,1,N,-73.958710,...,7.5,0.0,0.5,0.0,0.0,0.3,8.3,262,262,1
12748826,1,2015-01-10 18:16:49,2015-01-10 18:36:50,2,3.40,-73.947174,40.775917,1,N,-73.988899,...,15.5,0.0,0.5,1.5,0.0,0.3,17.8,262,262,1
12748858,2,2015-01-26 06:45:01,2015-01-26 07:04:05,1,4.35,-73.946953,40.779190,1,N,-73.990387,...,15.5,0.0,0.5,0.0,0.0,0.3,16.3,262,262,1
12748934,2,2015-01-09 15:04:27,2015-01-09 15:07:08,1,0.44,-73.949150,40.773232,1,N,-73.953262,...,4.0,0.0,0.5,0.8,0.0,0.3,5.6,262,262,1


In [35]:
taxi2015.query('PULocationID == 2')

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,...,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,PULocationID,DOLocationID,RatecodeID
261024,1,2015-01-23 21:42:46,2015-01-23 21:46:58,2,1.00,-73.834206,40.863976,1,N,-73.838539,...,5.50,0.5,0.5,0.00,0.00,0.3,6.80,2,2,1
706444,1,2015-01-11 01:05:10,2015-01-11 01:20:38,1,5.70,-73.848969,40.868534,1,N,-73.904060,...,19.00,0.5,0.5,4.05,0.00,0.3,24.35,2,2,1
752791,2,2015-01-19 00:13:59,2015-01-19 00:28:32,1,4.57,-73.857903,40.869251,1,N,-73.906776,...,15.50,0.5,0.5,1.20,0.00,0.3,18.00,2,2,1
1030339,2,2015-01-18 14:37:39,2015-01-18 15:58:39,3,12.53,-73.845070,40.862801,5,N,-73.885910,...,0.01,0.0,0.0,0.00,5.33,0.3,5.64,2,2,5
2902909,1,2015-01-01 00:34:42,2015-01-01 00:41:22,2,2.70,-73.858749,40.865551,1,N,-73.831215,...,10.00,0.5,0.5,0.00,0.00,0.0,11.30,2,2,1
3499198,1,2015-01-26 06:16:15,2015-01-26 06:34:06,1,4.70,-73.857101,40.869263,1,N,-73.873047,...,18.50,0.0,0.5,0.00,0.00,0.3,19.30,2,2,1
3599012,2,2015-01-01 10:49:41,2015-01-01 11:16:53,2,17.43,-73.843048,40.865501,1,N,-74.012444,...,48.00,0.0,0.5,0.00,5.33,0.3,54.13,2,2,1
3627154,2,2015-01-06 21:06:20,2015-01-06 21:15:19,1,2.26,-73.833138,40.863449,1,N,-73.830162,...,8.50,0.5,0.5,0.00,0.00,0.3,9.80,2,2,1
4148598,2,2015-01-24 04:25:56,2015-01-24 04:41:15,1,2.93,-73.859879,40.871281,1,N,-73.857094,...,11.00,0.5,0.5,0.00,0.00,0.3,12.30,2,2,1
5004763,1,2015-01-08 12:46:12,2015-01-08 13:09:41,0,10.90,-73.847313,40.865318,1,N,-73.950714,...,32.00,0.0,0.5,0.00,0.00,0.3,32.80,2,2,1


In [36]:
taxi2018.query('PULocationID == 2')

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
1231084,1,2018-01-06 12:56:22,2018-01-06 13:12:52,1,7.2,1,N,2,95,2,22.5,0.0,0.5,0.00,0.00,0.3,23.30
1773053,1,2018-01-08 14:58:22,2018-01-08 15:05:58,1,3.2,1,N,2,124,1,11.0,0.0,0.5,2.35,0.00,0.3,14.15
1903847,1,2018-01-08 23:13:57,2018-01-08 23:45:34,1,12.1,1,N,2,62,1,36.0,0.5,0.5,3.00,0.00,0.3,40.30
5372386,1,2018-01-20 15:02:38,2018-01-20 16:10:51,1,17.7,2,N,2,162,1,52.0,0.0,0.5,11.70,5.76,0.3,70.26


Sticking with `PULocationID == 2` to test that things are all working, we'll now get some numbers and use them for our next step

In [12]:
taxi2015['PULocationID'].count()

12748986

In [13]:
taxi2018['PULocationID'].count()

8759874

## Bringing Them All Together
If you wanted to include this as part of a larger clean up of Taxi data, you'd then concatenate this dataframe into a `dask_cudf` dataframe and delete its `cuDF` version, or convert it into arrow memory format and process it similar to how we did in the mortgage notebook.  For now, as we are only working on a couple of GBs, we'll concatonate in cuDF.

Before we do, let's change the `RateCodeID` in 2015 to match the `RatecodeID` column in 2018

In [25]:
taxi2015['RatecodeID'] = taxi2015['RateCodeID']
taxi2015.drop('RateCodeID')

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,...,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,PULocationID,DOLocationID,RatecodeID
0,2,2015-01-15 19:05:39,2015-01-15 19:23:42,1,1.59,-73.993896,40.750111,N,-73.974785,40.750618,...,12.0,1.0,0.5,3.25,0.0,0.3,17.05,185,185,1
1,1,2015-01-10 20:33:38,2015-01-10 20:53:28,1,3.30,-74.001648,40.724243,N,-73.994415,40.759109,...,14.5,0.5,0.5,2.00,0.0,0.3,17.80,212,212,1
2,1,2015-01-10 20:33:38,2015-01-10 20:43:41,1,1.80,-73.963341,40.802788,N,-73.951820,40.824413,...,9.5,0.5,0.5,0.00,0.0,0.3,10.80,166,166,1
3,1,2015-01-10 20:33:39,2015-01-10 20:35:31,1,0.50,-74.009087,40.713818,N,-74.004326,40.719986,...,3.5,0.5,0.5,0.00,0.0,0.3,4.80,230,230,1
4,1,2015-01-10 20:33:39,2015-01-10 20:52:58,1,3.00,-73.971176,40.762428,N,-74.004181,40.742653,...,15.0,0.5,0.5,0.00,0.0,0.3,16.30,162,162,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12748981,1,2015-01-10 19:01:44,2015-01-10 19:05:40,2,1.00,-73.951988,40.786217,N,-73.953735,40.775162,...,5.5,0.0,0.5,1.25,0.0,0.3,7.55,236,236,1
12748982,1,2015-01-10 19:01:44,2015-01-10 19:07:26,2,0.80,-73.982742,40.728184,N,-73.974976,40.720013,...,6.0,0.0,0.5,2.00,0.0,0.3,8.80,73,73,1
12748983,1,2015-01-10 19:01:44,2015-01-10 19:15:01,1,3.40,-73.979324,40.749550,N,-73.969101,40.787800,...,13.5,0.0,0.5,0.00,0.0,0.3,14.30,164,164,1
12748984,1,2015-01-10 19:01:44,2015-01-10 19:17:03,1,1.30,-73.999565,40.738483,N,-73.981819,40.737652,...,10.5,0.0,0.5,2.25,0.0,0.3,13.55,251,251,1


In [26]:
df = cudf.concat([taxi2018, taxi2015])

In [29]:
df.count()

VendorID                 21508860
tpep_pickup_datetime     21508860
tpep_dropoff_datetime    21508860
passenger_count          21508860
trip_distance            21508860
RatecodeID               21508860
store_and_fwd_flag       21508860
PULocationID             21508860
DOLocationID             21508860
payment_type             21508860
fare_amount              21508860
extra                    21508860
mta_tax                  21508860
tip_amount               21508860
tolls_amount             21508860
improvement_surcharge    21508857
total_amount             21508860
pickup_longitude         12748986
pickup_latitude          12748986
RateCodeID               12748986
dropoff_longitude        12748986
dropoff_latitude         12748986
dtype: int64

## Final check
Now to test to see if both years are present as expected

In [28]:
df.query('PULocationID == 2')

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,pickup_longitude,pickup_latitude,RateCodeID,dropoff_longitude,dropoff_latitude
1231084,1,2018-01-06 12:56:22,2018-01-06 13:12:52,1,7.20,1,N,2,95,2,...,0.5,0.00,0.00,0.3,23.30,null,null,null,null,null
1773053,1,2018-01-08 14:58:22,2018-01-08 15:05:58,1,3.20,1,N,2,124,1,...,0.5,2.35,0.00,0.3,14.15,null,null,null,null,null
1903847,1,2018-01-08 23:13:57,2018-01-08 23:45:34,1,12.10,1,N,2,62,1,...,0.5,3.00,0.00,0.3,40.30,null,null,null,null,null
5372386,1,2018-01-20 15:02:38,2018-01-20 16:10:51,1,17.70,2,N,2,162,1,...,0.5,11.70,5.76,0.3,70.26,null,null,null,null,null
261024,1,2015-01-23 21:42:46,2015-01-23 21:46:58,2,1.00,1,N,2,2,2,...,0.5,0.00,0.00,0.3,6.80,-73.83420563,40.86397552,1,-73.83853912,40.87530899
706444,1,2015-01-11 01:05:10,2015-01-11 01:20:38,1,5.70,1,N,2,2,1,...,0.5,4.05,0.00,0.3,24.35,-73.84896851,40.86853409,1,-73.90406036,40.90681458
752791,2,2015-01-19 00:13:59,2015-01-19 00:28:32,1,4.57,1,N,2,2,1,...,0.5,1.20,0.00,0.3,18.00,-73.85790253,40.86925125,1,-73.90677643,40.82606125
1030339,2,2015-01-18 14:37:39,2015-01-18 15:58:39,3,12.53,5,N,2,2,2,...,0.0,0.00,5.33,0.3,5.64,-73.84506989,40.8628006,5,-73.88591003,40.75167847
2902909,1,2015-01-01 00:34:42,2015-01-01 00:41:22,2,2.70,1,N,2,2,2,...,0.5,0.00,0.00,0.0,11.30,-73.85874939,40.86555099,1,-73.8312149,40.84788132
3499198,1,2015-01-26 06:16:15,2015-01-26 06:34:06,1,4.70,1,N,2,2,2,...,0.5,0.00,0.00,0.3,19.30,-73.85710144,40.8692627,1,-73.87304687,40.82069016


## Back To Your Workflow
So now you've seen how to use cuSpatial to clean and correlate your spatial data using the NYC taxi data.  You can now perform multi year analytics across the entire range of taxi datasets using your favorite RAPIDS libraries,